In [188]:
from pulp import *

num_members = 75
num_shifts = 4
num_days = 100
num_members_with_extra_shifts = 10 # Members indexed with 0 through x-1 will have extra shifts
num_extra_shifts = 2 # Average extra shift per member

all_days = range(num_days)
all_members = range(num_members)
all_shifts = range(1, num_shifts + 1)
start_day = 0  # Day of week mon-sun (0-6)

# List of all shifts that are forbidden (morning shifts in weekdays)
forbidden_shifts = [
    (d, s)
    for d in all_days
    for s in all_shifts
    if (((start_day + d) % 7) >= 0 and ((start_day + d) % 7) < 5) and s == 2
]

# List of all shifts considered bad
bad_shifts = [
    (d, 1) for d in all_days if ((start_day + d) % 7) != 5
]  # Every night shift
bad_shifts += [
    (d, s) for d in all_days for s in all_shifts if ((start_day + d) % 7) == 5
]  # All shifts on saturdays
bad_shifts += [
    (d, 2) for d in all_days if ((start_day + d) % 7) == 6
]  # Morning shift on sundays
bad_shifts += [
    (d, s)
    for d in all_days
    for s in all_shifts
    if ((start_day + d) % 7) == 4 and (s == 3 or s == 4)
]  # 3rd and 4th shift on friday

# Model variables
shifts = LpVariable.dicts(
    "shifts", (all_members, all_days, all_shifts), 0, 1, LpInteger
)
members = LpVariable.dicts("members", (all_members), 0, None, LpInteger)

# Define model
model = LpProblem("Scheduler", LpMaximize)
model += lpSum(members[si] for si in all_members)

# 4 shifts per day
for d in all_days:
    # Decreased number of shifts in weekdays
    if ((start_day + d) % 7) >= 0 and ((start_day + d) % 7) < 5:
        model += lpSum(shifts[m][d][r] for m in all_members for r in all_shifts) == 3
    else:
        model += lpSum(shifts[m][d][r] for m in all_members for r in all_shifts) == 4

# Distribute num of all shifts evenly
total_num_shifts = num_days * num_shifts - len(forbidden_shifts)
# Number of members without extra shift
num_members_normal = num_members - num_members_with_extra_shifts
# Calculate minimum shift per member without extra shift
min_shift_per_member = (
    total_num_shifts - num_members_with_extra_shifts * num_extra_shifts
) // (num_members_normal + num_members_with_extra_shifts)
# Calculate minimum shift per member with extra shifts
min_shift_per_extra_members = min_shift_per_member + num_extra_shifts

for n in all_members:
    # Differentiate between members with extra shifts and normal members
    if n < num_members_with_extra_shifts:
        model += (
            lpSum(shifts[n][d][s] for d in all_days for s in all_shifts)
            >= min_shift_per_extra_members
        )
        model += (
            lpSum(shifts[n][d][s] for d in all_days for s in all_shifts)
            <= min_shift_per_extra_members + 1
        )
    else:
        model += (
            lpSum(shifts[n][d][s] for d in all_days for s in all_shifts)
            >= min_shift_per_member
        )
        model += (
            lpSum(shifts[n][d][s] for d in all_days for s in all_shifts)
            <= min_shift_per_member + 1
        )

# Distribute bad shifts evenly
# This distribution is currently even,
# which means members with extra shifts will have lower rates of bad shifts (good or bad?)
total_num_bad_shifts = len(bad_shifts) // num_members
for n in all_members:
    if n < num_members_with_extra_shifts or True:
        model += (
            lpSum(
                shifts[n][d][s]
                for d in all_days
                for s in all_shifts
                if (d, s) in bad_shifts
            )
            >= total_num_bad_shifts
        )
        model += (
            lpSum(
                shifts[n][d][s]
                for d in all_days
                for s in all_shifts
                if (d, s) in bad_shifts
            )
            <= total_num_bad_shifts + 1
        )

# Only one shift per member per day
for m in members:
    for d in all_days:
        model += lpSum(shifts[m][d][r] for r in all_shifts) <= 1

# Every shift in the day must be occupied
for d in all_days:
    for r in all_shifts:
        if (d, r) not in forbidden_shifts:
            model += lpSum(shifts[m][d][r] for m in all_members) == 1
        else:
            # Exclude forbidden shifts
            model += lpSum(shifts[m][d][r] for m in all_members) == 0

# Number of shifts that a member has done
for m in members:
    model += (
        lpSum(shifts[m][d][r] for d in all_days for r in all_shifts) - members[m] == 0
    )

# Prevent two shifts in a row. Not needed in cases with many days
# for e in all_members:
#     for d in range(num_days - 1):
#         for s1 in all_shifts:
#             for s2 in all_shifts:
#                 transition = [shifts[e][d][s1], shifts[e][d + 1][s2]]
#                 model += lpSum(transition) <= 1


model.solve(PULP_CBC_CMD(msg=0))
print(LpStatus[model.status] + " solution")
if model.status == LpSolutionOptimal:
    print(f"Solved in {round(model.solutionTime, 1)}s")


Optimal solution
Solved in 1.3s


In [183]:
import pandas as pd
from IPython.display import display

# Display solution as table

print('Solution')
res = {str(day_names[d % 7])[0:2] + "_" + str(d): {} for d in all_days}

day_names = ["Mandag", "Tirsdag", "Onsdag", "Torsdag", "Fredag", "Lørdag", "Søndag"]

for vi in model.variables():
    if vi.name.split("_")[0] == 'shifts':
        person = int(vi.name.split("_")[1])
        day_int = int(vi.name.split("_")[2])
        role = int(vi.name.split("_")[3])

        day = str(day_names[day_int % 7])[0:2] + "_" + str(day_int)

        if vi.varValue:
            res[day][person] = role
        elif person not in res[day]:
            res[day][person] = "-"

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
display(pd.DataFrame(res))


Solution


,Ma_0,Ti_1,On_2,To_3,Fr_4,Lø_5,Sø_6,Ma_7,Ti_8,On_9,To_10,Fr_11,Lø_12,Sø_13,Ma_14,Ti_15,On_16,To_17,Fr_18,Lø_19,Sø_20,Ma_21,Ti_22,On_23,To_24,Fr_25,Lø_26,Sø_27,Ma_28,Ti_29,On_30,To_31,Fr_32,Lø_33,Sø_34,Ma_35,Ti_36,On_37,To_38,Fr_39,Lø_40,Sø_41,Ma_42,Ti_43,On_44,To_45,Fr_46,Lø_47,Sø_48,Ma_49,Ti_50,On_51,To_52,Fr_53,Lø_54,Sø_55,Ma_56,Ti_57,On_58,To_59,Fr_60,Lø_61,Sø_62,Ma_63,Ti_64,On_65,To_66,Fr_67,Lø_68,Sø_69,Ma_70,Ti_71,On_72,To_73,Fr_74,Lø_75,Sø_76,Ma_77,Ti_78,On_79,To_80,Fr_81,Lø_82,Sø_83,Ma_84,Ti_85,On_86,To_87,Fr_88,Lø_89,Sø_90,Ma_91,Ti_92,On_93,To_94,Fr_95,Lø_96,Sø_97,Ma_98,Ti_99
0,-,4,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,4,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,4,-,3,-,-,-,-,-,-,-,-,-,-,-,2,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,4,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
10,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,3,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,3,-,-,-,-,-,4,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1,-,-
11,-,-,3,-,3,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,2,-,-,-,3,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
12,-,-,-,-,-,-,4,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1,-,-,-,-,-,-,-,-,-,-,-,1,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
13,-,-,-,-,-,-,3,-,-,4,-,-,-,-,-,-,-,-,-,-,-,-,1,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,4,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
14,-,-,-,-,-,-,-,-,-,-,1,-,3,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,4,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,3,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
15,-,-,-,-,-,-,-,-,-,-,-,3,-,3,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,4,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1,-,-,-,-,-,-,-,-,-,3,-,-,-,-,-,-,-,-,-
16,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,3,-,-,2,-,-,3,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1,-,-,-,-,-,-,-,-
17,-,-,-,-,-,-,-,1,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,3,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,4,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,3,-,-,-,-,-,-,-,-,-,-,-,-
18,-,-,1,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,3,-,-,3,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,3,-,-,-,-,-,-,3,-,-,-,-


In [167]:
# Test shift distribution

test_bad_shifts = False
members = {}

for day, member_shift in res.items():
    for member, shift in member_shift.items():
        if shift != "-" and ((int(day.split("_")[1]), shift) in bad_shifts or not test_bad_shifts):
            if member in members:
                members[member] += 1
            else:
                members[member] = 1

print({k: v for k, v in sorted(members.items(), key=lambda item: item[0])})

{0: 6, 1: 6, 2: 6, 3: 6, 4: 6, 5: 6, 6: 6, 7: 7, 8: 6, 9: 6, 10: 5, 11: 5, 12: 5, 13: 4, 14: 5, 15: 5, 16: 5, 17: 5, 18: 4, 19: 4, 20: 5, 21: 4, 22: 5, 23: 4, 24: 5, 25: 5, 26: 4, 27: 4, 28: 4, 29: 4, 30: 5, 31: 4, 32: 5, 33: 4, 34: 5, 35: 4, 36: 4, 37: 5, 38: 4, 39: 4}


In [143]:
# Find bad shifts

bad_shifts = [(d, 1) for d in all_days if ((start_day + d) % 7) != 5] 
bad_shifts += [(d, s) for d in all_days for s in all_shifts if ((start_day + d) % 7) == 5]
bad_shifts += [(d, 2) for d in all_days if ((start_day + d) % 7) == 6]
bad_shifts += [(d, s) for d in all_days for s in all_shifts if ((start_day + d) % 7) == 4 and (s == 3 or s == 4)]

print(len(bad_shifts) // num_members)

3


In [133]:
# Test shift distribution algebra

total_num_shifts = num_days * num_shifts - len(forbidden_shifts)
num_members_normal = num_members - num_members_with_extra_shifts
num_extra_shifts = 1
min_shift_per_member = ((total_num_shifts - num_members_with_extra_shifts*num_extra_shifts) // (num_members_normal + num_members_with_extra_shifts))
min_shift_per_extra_members = min_shift_per_member + num_extra_shifts

print(min_shift_per_member*num_members_normal+min_shift_per_extra_members*num_members_with_extra_shifts)
print((min_shift_per_member+1)*num_members_normal+(min_shift_per_extra_members+1)*num_members_with_extra_shifts)
print(total_num_shifts)

185
220
196
